# PRO3515 - PFSP LS


## Dupla 9

### Nomes: 
### Lucas Hideki Takeuchi Okamura NUSP:  9274315
### Thales Arantes Kerche Nunes   NUSP: 10769372


Importando as bibliotecas necessárias

In [1]:
import xlwings as xw
import random
import math

Definindo a classe Job que usaremos no algoritmo

In [2]:
class Job:
    def __init__(self,i,pi,di):
        self.i=i     # número do job, pela ordem de chegada
        self.p=pi     # processing time
        self.p_total=sum(self.p)
        self.d=di     # due date
        self.C=0     # completion time
        self.T=0     # tardiness
    
    def __str__(self):
        return 'Job #{}, p = {}'.format(self.i, self.p)

### Reutilizamos a função disponibilizada pelo professor, com alguns ajustes para ler o número de máquinas

In [3]:
def leInst(sheet_num):
    pl1 = wb1.sheets[sheet_num]
    m=int(pl1.range('B5').value)     # converter em inteiro
    n=int(pl1.range('B6').value)     # converter em inteiro
    tab1=pl1.range('B11').options(expand='table', numbers=int).value     # ler como tabela e inteiros
    print()
#     print('n =',n)
#     print('m =',m)
#     print('dados =',tab1)
    jobs=[]
    for i in range(n):
        pi=tab1[i]
        jobs.append(Job(i,pi,0))
    
    return m,n,jobs

Também reutilizamos a de gravar, novamente, com alguns ajustes

Função que permite calcular o tempo que os jobs se completam

In [4]:
def calcCmax(m,jobs):     # esta função calcula os tempos de conclusão dos jobs nas m máquinas
    n=len(jobs)
    c=[[0 for i in range(m)] for j in range(n)]
    p=[job.p for job in jobs]
    
    # primeiro job
    c[0][0]=p[0][0]     # primeira máquina
    for k in range(1,m):     # próximas máquinas
        c[0][k]=c[0][k-1]+p[0][k]
        
    # próximos jobs
    for j in range(1,n):
        c[j][0]=c[j-1][0]+p[j][0]     # primeira máquina
        for k in range(1,m):     # próximas máquinas
            c[j][k]=max(c[j][k-1],c[j-1][k])+p[j][k]
            
    j=0
    for job in jobs:
        job.C=c[j]
        j+=1
    
    return c[-1][-1]

In [5]:
def permutar(jobs, new_job, m):

    bestCmax = 9999999999999

    for index in range(len(jobs)+1):

        aux_jobs = jobs[:]

        aux_jobs.insert(index, new_job)

        Cmax = calcCmax(m, aux_jobs)

        if Cmax < bestCmax:
            final_jobs = aux_jobs
            bestCmax = Cmax
    
    bestCmax = calcCmax(m, final_jobs)

    return final_jobs, bestCmax

In [6]:
def schedNEH(jobs, m):

    jobs.sort(key = lambda job: job.p_total, reverse=True)

    jobs_permutacao = jobs[:1]

    for i in range(1, len(jobs)):

        jobs_permutacao, Cmax = permutar(jobs_permutacao, jobs[i], m)
    
    return jobs_permutacao,Cmax

In [7]:
def schedLS(jobs, bestCmax, m):

    melhoria = True
    aux_jobs = jobs[:]
    while melhoria:
        melhoria = False
        jotas = list(range(len(aux_jobs)))
        # random.shuffle(jotas)

        for j1 in jotas:
            job1 = aux_jobs.pop(j1)
            bestPos = j1

            for j2 in range(len(aux_jobs)+1):
                aux_jobs.insert(j2, job1)
                Cmax = calcCmax(m, aux_jobs)

                if Cmax < bestCmax:
                    bestPos = j2
                    melhoria = True
                    bestCmax = Cmax
                    # break
                aux_jobs.pop(j2)
            # if melhoria:
            #     break
            aux_jobs.insert(bestPos, job1)

    
    return aux_jobs,Cmax

Função que calcula a Temperatura com a fórmula presente no artigo:

In [8]:
def Temperatura(T: float, jobs: list, n: int = 20, m: int = 5):
    return (T*sum([job.p_total for job in jobs])/(10*n*m))

Função que acha a melhor posição para um job a ser inserido em uma fila de jobs:

In [9]:
def findBestPosition(aux_jobs: list, new_job: Job, m: int) -> (list, int):
    
    bestCmax = float('inf')
    for ind in range(len(aux_jobs)+1):
        aux_jobs.insert(ind, new_job)
        Cmax = calcCmax(m, aux_jobs)

        if Cmax < bestCmax:
            bestPos = ind
            bestCmax = Cmax
        aux_jobs.pop(ind)
    aux_jobs.insert(bestPos, new_job)

    return aux_jobs, bestCmax

### Função principal `schedIG`

In [10]:
def schedIG(jobs: list, bestCmax: int, T: float = 0.5, d: int = 3, m: int = 5) -> (list, int):

    jobs_b = jobs[:]
    for _ in range(100):
        
        jobs_linha = jobs[:]
        retirados = random.sample(jobs_linha, d)
        for item in retirados:
            jobs_linha.remove(item)
        for new_job in retirados:
            jobs_linha, newCmax = findBestPosition(jobs_linha, new_job, m)
        
        if newCmax < calcCmax(m, jobs) or random.random() <= math.exp(-(newCmax-calcCmax(m, jobs))/Temperatura(T, jobs)):
            jobs = jobs_linha[:]
        
    return jobs, calcCmax(m, jobs)

## Principal

Caso base pra comprovar o funcionamento do algoritmo implementado

In [11]:
tab1 = [[6,3],[1,3],[8,6],[2,5],[4,6]]
jobs=[]
for i in range(len(tab1)):
    pi=tab1[i]
    jobs.append(Job(i,pi,0))

for _ in range (3):
    jobs, Cmax = schedNEH(jobs, m=2)
    jobs, Cmax = schedLS(jobs, Cmax, m=2)
    jobs, Cmax = schedIG(jobs, Cmax, m=2)

    print([job.i+1 for job in jobs])
    print([job.p for job in jobs])
    print('Cmax = ', Cmax)

[2, 4, 5, 3, 1]
[[1, 3], [2, 5], [4, 6], [8, 6], [6, 3]]
Cmax =  24
[2, 4, 5, 3, 1]
[[1, 3], [2, 5], [4, 6], [8, 6], [6, 3]]
Cmax =  24
[2, 4, 5, 3, 1]
[[1, 3], [2, 5], [4, 6], [8, 6], [6, 3]]
Cmax =  24


Abre os dois arquivos que vão ser usados

In [12]:
wb1 = xw.Book("Dupla 09 - Desafio 7 - Instâncias.xlsx")

Roda todos os testes:

In [13]:
d_list = [2, 3, 5, 8]
T_list = [0.000001, 0.1, 0.2, 0.5]
Cmax_list = []
for i in range(5):
    for d in d_list:
        for T in T_list:
            for k in range (3):
                m,n,jobs=leInst(i)
                jobs, Cmax = schedNEH(jobs, m=5)
                jobs, Cmax = schedLS(jobs, Cmax, m=5)
                jobs, Cmax = schedIG(jobs, Cmax, m=5, d=d, T=T)
                jobs, Cmax = schedIG(jobs, Cmax)
                Cmax_list.append(Cmax)
                print(f"Cmax - i={i} - d={d} - T={T} - k={k} = {Cmax}")


Cmax - i=0 - d=2 - T=1e-06 - k=0 = 1199

Cmax - i=0 - d=2 - T=1e-06 - k=1 = 1195

Cmax - i=0 - d=2 - T=1e-06 - k=2 = 1200

Cmax - i=0 - d=2 - T=0.1 - k=0 = 1195

Cmax - i=0 - d=2 - T=0.1 - k=1 = 1199

Cmax - i=0 - d=2 - T=0.1 - k=2 = 1198

Cmax - i=0 - d=2 - T=0.2 - k=0 = 1195

Cmax - i=0 - d=2 - T=0.2 - k=1 = 1199

Cmax - i=0 - d=2 - T=0.2 - k=2 = 1199

Cmax - i=0 - d=2 - T=0.5 - k=0 = 1199

Cmax - i=0 - d=2 - T=0.5 - k=1 = 1198

Cmax - i=0 - d=2 - T=0.5 - k=2 = 1195

Cmax - i=0 - d=3 - T=1e-06 - k=0 = 1199

Cmax - i=0 - d=3 - T=1e-06 - k=1 = 1199

Cmax - i=0 - d=3 - T=1e-06 - k=2 = 1199

Cmax - i=0 - d=3 - T=0.1 - k=0 = 1199

Cmax - i=0 - d=3 - T=0.1 - k=1 = 1199

Cmax - i=0 - d=3 - T=0.1 - k=2 = 1195

Cmax - i=0 - d=3 - T=0.2 - k=0 = 1196

Cmax - i=0 - d=3 - T=0.2 - k=1 = 1198

Cmax - i=0 - d=3 - T=0.2 - k=2 = 1195

Cmax - i=0 - d=3 - T=0.5 - k=0 = 1209

Cmax - i=0 - d=3 - T=0.5 - k=1 = 1196

Cmax - i=0 - d=3 - T=0.5 - k=2 = 1214

Cmax - i=0 - d=5 - T=1e-06 - k=0 = 1199

Cmax - i=0

Fecha o workbook de Dados

In [121]:
wb1.close()